### First we gather the data

In [33]:
import numpy as np

point_image_1=[5.31076477e+02, 2.40788101e+02] #upperleft disk
point_image_2=[1.29228992e+03, 2.86447327e+02] # upper right
point_image_3=[1.25122437e+03, 7.30169861e+02] # bottom right
point_image_4=[4.97040863e+02, 8.80103821e+02] # bottom left
point_image_5=[5.32581970e+02, 2.42284805e+02] #Initial point to close the process 

points_source=np.array([point_image_1,point_image_2,point_image_3,point_image_4])

#All the points are measured in meters
point_monde_1=[0,0,1]
point_monde_2=[.217,0,1]
point_monde_3=[.222,.118,1]
point_monde_4=[0,.118,1]
point_monde_5=[0,0,1]

points_target=np.array([point_monde_1,point_monde_2,point_monde_3,point_monde_4,point_monde_5])
print(points_source.shape)
a=np.take(points_target,0,axis=1)
print(points_source)
print(points_source[1][0])

(4, 2)
[[ 531.076477  240.788101]
 [1292.28992   286.447327]
 [1251.22437   730.169861]
 [ 497.040863  880.103821]]
1292.28992


### Second we normalize the data

Apparently the normalization is an essential step, indeed we can understand that untuitvely (the 2D homographics depends on the coordinate frame in which points are expressed. NOTE : Some  coordiante systems are in some way better than others.

Without normalization typical image points x, x' are of the order( x,y,w)= (100,100,1) i.e x,y are much larger than w. In A the entries xx', xy',yx',yy' will be of order 10^4, entries xw', yw' of order 10^2, and entries ww' will be unity. Replacing A by A tilde means that some entries are increased and others decreased such that the square sum of the differences of these changes is minimal(and the resulting matrix has rank 8). However, and this is the key point, increasing the term ww' by 100 means a huge change in the image points, whereas increasing the term xx' by 100 means only a slight change. This is the reason why all entries in A must have similar magnitude and why nomralization is essential.

  The effect of nomalization is related to the condition number of the set of DLT equations, or more precisely the ration d/dn-1 of the first to the second-last singular value of the equation matrix A. This point is investigated in more detail in [ref]. The normalization is an essential step !
  
  Algorithm
  
  (i) Normalization of x : Compute a similarity transformation T, consisting of a translation and sclaing, that takes points x_i to a new set of points x_i tilde such that the centroid of the points x_itilde is the coordinate origin(0,0), and their average distance from the origin is squareroot(2)
  
  (ii) Normalization of x' : Compute a similar transformation T' for the points in the second image, transforming points xi'to x_itilde'
  (iii) DLT: apply DLT to the correspondences x_itilde <-> x_itilde' to obtain a homography Htilde
  (iv) Denormalization : SET H=T'^-1*Htilde*T
  

#### Build the T and T' matrices

In [18]:
# First we require the mean values of u_i and v_i (aka x and y of each point)

def ComputeArrayOfAverageOfEachCoordinate(pointsArray):
    return np.average(pointsArray, axis=0)


def ComputeScalarNormalization(pointsArray):
    n=len(pointsArray)
    
    arrayOfAverageOfEachCoordinate=ComputeArrayOfAverageOfEachCoordinate(pointsArray)
    
    averageX=arrayOfAverageOfEachCoordinate[0]
    averageY=arrayOfAverageOfEachCoordinate[1]
   
    arrayofXcoordinates=np.take(points_target,0,axis=1)
    arrayofYcoordinates=np.take(points_target,1,axis=1)
    
    # The below two lines will provide us with generators
    arrayFirstElementLeastSquare=((arrayofXcoordinates[i]-averageX)**2 for i in range(n-1))
    arraySecondElementLeastSquare=((arrayofYcoordinates[i]-averageY)**2 for i in range(n-1))
    
    
    #The below two lines will convert the generators to npArrays by making them a List first
    arrayFirstElementLeastSquare = np.array(list(arrayFirstElementLeastSquare))
    arraySecondElementLeastSquare = np.array(list(arraySecondElementLeastSquare))
    
    # The above was done so that we can apply :
    arrayAdditionRootElement=np.sqrt(np.add(arrayFirstElementLeastSquare, arraySecondElementLeastSquare)) 
    
    sumDenominator=sum(arrayAdditionRootElement)
    
    ScaleFactorNormalizationArrayPoints= 2**(1/2)*n/(sumDenominator)
    return ScaleFactorNormalizationArrayPoints,averageX,averageY
    

ScaleFactorNormalizationTargetPoints,averageTargetX,averageTargetY=ComputeScalarNormalization(points_source)

# After having the averages and the Scale Factor related to the Normalization we will build 
def BuildNormalizationMatrix(ScaleFactorNormalizationArrayPoints,averageX,averageY):
    
    NormalizationMatrix_T= np.array([
        [1,0,-averageX],
        [0,1, -averageY],
        [0,0, 1/ScaleFactorNormalizationArrayPoints]
     ])
        
    return ScaleFactorNormalizationArrayPoints*NormalizationMatrix_T

def MultiplyMatrixArray(matrix,arrayPoints):
    resultingArray=(np.matmul(matrix,arrayPoints[i]) for i in range(len(arrayPoints)))
    return  np.array(list(resultingArray))
    




In [19]:
#Now we normalize the data
scaleFactorNormalizationTargetPoints,averageTargetX,averageTargetY=ComputeScalarNormalization(points_target)
normalizationMatrix_TargetPoints_T=BuildNormalizationMatrix(scaleFactorNormalizationTargetPoints,averageTargetX,averageTargetY)

scaleFactorNormalizationSourcePoints,averageSourceX,averageSourceY=ComputeScalarNormalization(points_source)
normalizationMatrix_SourcePoints_T=BuildNormalizationMatrix(scaleFactorNormalizationSourcePoints,averageSourceX,averageSourceY)


print(normalizationMatrix_TargetPoints_T)

print(normalizationMatrix_SourcePoints_T)
# After having the averages and the Scale Factor related to the Normalization we will build 

NormalizedPoints_target=MultiplyMatrixArray(normalizationMatrix_TargetPoints_T,points_target)

NormalizedPoints_source=MultiplyMatrixArray(normalizationMatrix_SourcePoints_T,points_source)

print(NormalizedPoints_target)

print(NormalizedPoints_source)

[[14.09266705  0.         -1.23733617]
 [ 0.         14.09266705 -0.66517388]
 [ 0.          0.          1.        ]]
[[ 0.0018633   0.         -1.52947939]
 [ 0.          0.0018633  -0.88685583]
 [ 0.          0.          1.        ]]
[[-1.23733617 -0.66517388  1.        ]
 [ 1.82077258 -0.66517388  1.        ]
 [ 1.89123592  0.99776083  1.        ]
 [-1.23733617  0.99776083  1.        ]
 [-1.23733617 -0.66517388  1.        ]]
[[-0.53992255 -0.43819443  1.        ]
 [ 0.87844938 -0.35311742  1.        ]
 [ 0.80193178  0.47367248  1.        ]
 [-0.60334124  0.753045    1.        ]
 [-0.53711736 -0.43540562  1.        ]]


In [20]:
def find_homography(points_source, points_target):
    A  = construct_A(points_source, points_target)#construct our A matrix using the function 
    u, s, vh = np.linalg.svd(A, full_matrices=True)#compute the svd 
    
    # Solution to H is the last column of V, or last row of V transpose
    homography = vh[-1].reshape((3,3)) # the reshape method gives a new shape to an array without changing its data
                                       #In this case the last column(vh[-1] represents the element before the first one)
                                       #matrix vh[-1] is reshaped as 3*3 matrix while originally being a vector
                                       #with a length of 9
    
    return homography/homography[2,2]#normalize H

def construct_A(points_source, points_target):
    assert points_source.shape == points_target.shape, "Shape does not match" #verify that the sizes are the same (3D)
    num_points = points_source.shape[0] #Extract the number of points from the points source matrix
                                        #in this case : shape[0]= length of the matrix => the number of points
    matrices = [] 
    for i in range(num_points):
        partial_A = construct_A_partial(points_source[i], points_target[i]) #Construct a "portion" of the A matrix (which
                                                                            #means that it constructs the A matrix for 
                                                                            #one point)
        matrices.append(partial_A) # we add the resulting matrix to the emmpty matrix
    return np.concatenate(matrices, axis=0)

def construct_A_partial(point_source, point_target):
    x, y, z = point_source[0], point_source[1], 1
    x_t, y_t, z_t = point_target[0], point_target[1], 1
    #A_partial represents the A matrix for one point 
    # Even after changing the hereafter line nothing has changed 
    A_partial = np.array([
        [0, 0, 0, -z_t*x, -z_t*y, -z_t*z, y_t*x, y_t*y, y_t*z],
        [z_t*x, z_t*y, z_t*z, 0, 0, 0, -x_t*x, -x_t*y, -x_t*z]
    ])
    return A_partial

In [21]:
#The following homography is a normalized one
normalized_Homography=find_homography(NormalizedPoints_target, NormalizedPoints_source)

In [22]:
#We denormalize H
Reversed_Tsource=np.linalg.inv(normalizationMatrix_SourcePoints_T)
reversed_Tsource_mult_NormHomography=np.matmul(Reversed_Tsource,normalized_Homography)
H=np.matmul(reversed_Tsource_mult_NormHomography,normalizationMatrix_TargetPoints_T)
point_monde_test1=[1197,176,1]
point_monde_test2=[950,750,1]

point_image_test=np.matmul(H,point_monde_test2) #The image point is the on located in the photo taken by the camera 
print(H)

[[ 5.13408271e+03 -4.66480364e+01  4.41415970e+02]
 [ 6.65070928e+02  4.84228265e+03  2.00473490e+02]
 [ 1.72206605e+00  3.98424386e-01  8.29996969e-01]]


## Building the K matrix

In [29]:
# Building the K matrix
#From the calibration of the camera we've done using matlab we found that 
#all length units are in mm 
Focal_length_X = 3146.7211
Focal_length_Y =3150.2599
Principal_point_X=1960.7933
Principal_point_Y=1503.0496

K=np.zeros((3,3))
K[0][0]=Focal_length_X
K[1][1]=Focal_length_Y
K[0][2]=Principal_point_X
K[1][2]=Principal_point_Y
K[2][2]=1

print(K)
print(K[2])


[[3.1467211e+03 0.0000000e+00 1.9607933e+03]
 [0.0000000e+00 3.1502599e+03 1.5030496e+03]
 [0.0000000e+00 0.0000000e+00 1.0000000e+00]]
[0. 0. 1.]


## Building the 2D components of the projection matrix

In [24]:
#reverse the K matrix to get the results of K^-1*H
reverse_K = np.linalg.inv(K)



homography_mult_Reverse_K=np.matmul(reverse_K,H)
print(homography_mult_Reverse_K)
R1_tilde=np.array(homography_mult_Reverse_K[:,0]) #The first column
R2_tilde=np.array(homography_mult_Reverse_K[:,1]) #The second column
Trans_tilde=np.array(homography_mult_Reverse_K[:,2]) #The third column

#promoting the arrays from row to column ones
R1_tilde = R1_tilde[:, np.newaxis]
R2_tilde = R2_tilde[:, np.newaxis]
Trans_tilde = Trans_tilde[:, np.newaxis]


[[ 0.55850744 -0.2630916  -0.37691187]
 [-0.61051463  1.34700982 -0.33237039]
 [ 1.72206605  0.39842439  0.82999697]]


## Building the 2D components of the projection matrix

In [25]:
print(R1_tilde)

print(R2_tilde)

print(Trans_tilde)

[[ 0.55850744]
 [-0.61051463]
 [ 1.72206605]]
[[-0.2630916 ]
 [ 1.34700982]
 [ 0.39842439]]
[[-0.37691187]
 [-0.33237039]
 [ 0.82999697]]


In [26]:

firstColumnRotationaMatrix=R1_tilde#which is in reality a times R1_tilde representing R1
secondColumnRotationaMatrix=R2_tilde#which is in reality a times R2_tilde representing R2
#set the third parameter axis=0 which tells cross that the vectors are defined along the first axis, rather than the last axis
thirdColumnRotationaMatrix=np.cross(R1_tilde,R2_tilde,axis=0)#which is in reality (a^2 times R1_tilde cross R2_tilde) representing R3

print(firstColumnRotationaMatrix)
print(secondColumnRotationaMatrix)
print(thirdColumnRotationaMatrix)

rotationalMatrixTilde=np.concatenate([firstColumnRotationaMatrix,secondColumnRotationaMatrix,thirdColumnRotationaMatrix], axis=1)  

print(np.array(rotationalMatrixTilde))


firstSolutionScaleFactor= np.linalg.det(rotationalMatrixTilde)**(-1/4)

secondSolutionScaleFactor=-firstSolutionScaleFactor
print(firstSolutionScaleFactor)
print(secondSolutionScaleFactor) #other possible solution 
#Hyptothesis on the value of, the value is for points that are in meter 

[[ 0.55850744]
 [-0.61051463]
 [ 1.72206605]]
[[-0.2630916 ]
 [ 1.34700982]
 [ 0.39842439]]
[[-2.56288381]
 [-0.6755841 ]
 [ 0.59169373]]
[[ 0.55850744 -0.2630916  -2.56288381]
 [-0.61051463  1.34700982 -0.6755841 ]
 [ 1.72206605  0.39842439  0.59169373]]
0.6068217399359956
-0.6068217399359956


### Building the projection matrix having the FIRST SOLUTION

In [27]:
fourthColumnProjectionMatrix=firstSolutionScaleFactor*(Trans_tilde)
print(fourthColumnProjectionMatrix)

rotationalMatrixProjection=firstSolutionScaleFactor*rotationalMatrixTilde
print(rotationalMatrixProjection)
rotationalMatrixProjection[:,2]*=firstSolutionScaleFactor

print(rotationalMatrixProjection)
projectionMatrixFirstSolution=np.hstack((rotationalMatrixProjection,fourthColumnProjectionMatrix) )
print(np.linalg.det(rotationalMatrixProjection))


#append the last row representing the 0 0 0 1
homogenousLine=np.array([0,0,0,1])

projectionMatrixFirstSolution=np.vstack([projectionMatrixFirstSolution,homogenousLine])

print(projectionMatrixFirstSolution)

[[-0.22871832]
 [-0.20168958]
 [ 0.50366021]]
[[ 0.33891445 -0.15964971 -1.55521361]
 [-0.37047355  0.81739485 -0.40995912]
 [ 1.04498712  0.24177258  0.35905262]]
[[ 0.33891445 -0.15964971 -0.94373743]
 [-0.37047355  0.81739485 -0.24877211]
 [ 1.04498712  0.24177258  0.21788093]]
1.0000000000000002
[[ 0.33891445 -0.15964971 -0.94373743 -0.22871832]
 [-0.37047355  0.81739485 -0.24877211 -0.20168958]
 [ 1.04498712  0.24177258  0.21788093  0.50366021]
 [ 0.          0.          0.          1.        ]]


### Building the projection matrix having the SECOND SOLUTION

In [28]:
#Since the determinent is negative the first solution is the correct onefourthColumnProjectionMatrix=secondSolutionScaleFactor*(Trans_tilde)


rotationalMatrixProjection=secondSolutionScaleFactor*rotationalMatrixTilde
rotationalMatrixProjection[:,2]*=firstSolutionScaleFactor
print(np.linalg.det(rotationalMatrixProjection))

projectionMatrixSecondSolution=np.hstack((rotationalMatrixProjection,fourthColumnProjectionMatrix) )

#Since the determinent is negative the first solution is the correct one


projectionMatrixSecondSolution=np.vstack([projectionMatrixSecondSolution,homogenousLine])
print(projectionMatrixSecondSolution)

-1.0000000000000002
[[-0.33891445  0.15964971  0.94373743  0.22871832]
 [ 0.37047355 -0.81739485  0.24877211  0.20168958]
 [-1.04498712 -0.24177258 -0.21788093 -0.50366021]
 [ 0.          0.          0.          1.        ]]


In [55]:
print(cubePoints(points_source,H,projectionMatrixFirstSolution,2))

[[5.31076477e+02 1.29228992e+03 1.25122437e+03 4.97040863e+02]
 [2.40788101e+02 2.86447327e+02 7.30169861e+02 8.80103821e+02]
 [1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00]]
[[2.71579968e+06 6.62180255e+06 6.39026983e+06 2.51123520e+06]
 [1.51936804e+06 2.24672385e+06 4.36804227e+06 4.59247936e+06]
 [1.01131462e+03 2.34036620e+03 2.44643849e+03 1.20742202e+03]]
[1011.31462118 2340.36620014 2446.43848846 1207.42201859]
[[2.68541523e+03 2.82938736e+03 2.61207051e+03 2.07983221e+03]
 [1.50236930e+03 9.59988164e+02 1.78546990e+03 3.80354117e+03]
 [1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00]]
[[ 2.68541523e+03  2.82938736e+03  2.61207051e+03  2.07983221e+03]
 [ 1.50236930e+03  9.59988164e+02  1.78546990e+03  3.80354117e+03]
 [-2.00000000e+00 -2.00000000e+00 -2.00000000e+00 -2.00000000e+00]
 [ 1.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00]]
[[ 6.71931974e+02  8.07317200e+02  6.01877464e+02  9.93097261e+01]
 [ 2.33449461e+02 -2.63227954e+02  4